# Funções utilizadas

## Funções para bombas
    1. reynolds: Calcula o valor do número de Reynolds para escoamentos internos
    2. npsh_d: Calcula o NPSH disponível de acordo com os parâmetros
    3. fator_f: Calcula o fator de atrito, podendo ser definido um valor arbitrário para a transição
    4. perda_carga: Calcula a perda de carga de acordo com perdas maiores e perdas menores do sistema
    5. calc_pot: Calcula e retorna o head (em m) e a potência da bomba

In [1]:
#FUNÇÕES ESPECÍFICAS:
def reynolds(velocidade, diametro, viscosidade=0.00089008, densidade=997) -> float:
    return densidade*velocidade*diametro/viscosidade #calcula e retorna o número de reynolds de acordo com a fórmula

def npsh_d(altura_succao, pressao_succao, pressao_vapor, perdas, densidade=997, g=9.81) -> float:
    #calcula e retorna o NPSH de acordo com a fórmula
    return  altura_succao + (pressao_succao - pressao_vapor)/(densidade*g) - perdas

def fator_f(re, eD, transicao=2500) -> float:
    from numpy import log10, sqrt
    from scipy.optimize import minimize
    if re <= transicao: #região laminar
        resp_f = 64/re

    elif re >= transicao: #região turbulenta
        def func(resp_f_aux): #a função usa a equação de Colebrook
            resp1 = 1/sqrt(resp_f_aux)
            resp2 = -2*log10(eD/3.7 + 2.51/(re*sqrt(resp_f_aux)))
            return (resp1 - resp2)**2 #o retorno é a comparação entre os dois lados da igualdade da equação
        resp_f = minimize(func, 1e-8).x[0]
        #o resultado é quando a diferença quadrada dos dois lados é próxima de zero
        
    return resp_f

def perda_carga(vazao_v, diametro, rugosidade, comprimento, LeD=0, K=0, viscosidade=0.00089008, densidade=997, g=9.81) -> float:
    from numpy import pi
    velocidade = vazao_v / (pi/4 * diametro**2) #calcula a velocidade
    
    eD = rugosidade/diametro
    re = reynolds(velocidade, diametro, viscosidade=viscosidade, densidade=densidade)
    f = fator_f(re, eD)
    
    return velocidade**2/(2*g) * (K + f*LeD + f*comprimento/diametro)

def calc_pot(vazao_v, altura=(0., 0.), pressao=(0., 0.), perdas=0, densidade=997, g=9.81, eficiencia=1.) -> list:
    head_bomba = (altura[1] - altura[0]) +\
                 (pressao[1] - pressao[0])/(densidade*g) +\
                 perdas #calcula o Head requerido em unidade de distâcia

    pot_ideal = head_bomba * vazao_v * densidade * g #calcula a potência da bomba sem considerar eficiência

    return [pot_ideal/eficiencia, head_bomba]

## Funções para compressores
    1. compress_pot: Calcula a potência do compressor de acordo com os parâmetros
    2. compress_est: Calcula o número de estágios para dadas pressões e razão de compressão
As funções a seguir resolvem as equações a seguir de acordo com uma variável defininda como **'x'** que é definida arbitrariamente e pode ser qualquer um dos componentes da equação

    3. compressor
\begin{align}
H_c - \left(\frac{z_a + z_b}{2}\right) \frac{R T_a}{m} \left(\frac{(p_b/p_a)^{ \frac{\gamma - 1}{n\gamma} } - 1}{\frac{\gamma - 1}{n\gamma}}\right) = 0
\end{align}

    4. compress_ts
\begin{align}
T_b - T_a \left(1 + \frac{ (p_b/p_a)^{ \frac{\gamma - 1}{\gamma} } - 1}{\eta}\right) = 0
\end{align}

In [8]:
def compressor(H_compressor=0, pressao=(101235., 101235.), compressibilidade=(1., 1.), temperatura=298, coef_adiabatico=5/3, massa_molar=1, R=8.314, estagios=1) -> float:
    #ESSA FUNÇÃO SÓ PERMITE UMA VÁRIAVEL DE CADA VEZ, CASO SEJAM COLOCADOS DOIS X A FUNÇÃO IRÁ FALHAR (isso é intencional)
    from sympy import symbols, solveset
    variaveis = [H_compressor, *pressao, *compressibilidade, temperatura, coef_adiabatico, massa_molar, estagios] #lista de variaveis
              # [0, (1, 2), (3, 4), 5, 6, 7, 8] ordem dos valores da lista de variaveis
    x = symbols('x')
    
    cont = 0
    for i, j in enumerate(variaveis): #procura qual é a variável e qual a sua posição na lista de variáveis
        if j == 'x': variaveis[i], cont = x, cont+1
    
    if cont == 1: #checa se existe apenas uma variável
        function =  variaveis[0] - (variaveis[3] + variaveis[4])/2 *\
                                   (R * variaveis[5])/variaveis[7] *\
                                   ((variaveis[8]*variaveis[6])/(variaveis[6]-1)) *\
                                   ((variaveis[2]/variaveis[1])**((variaveis[6]-1)/(variaveis[8]*variaveis[6])) - 1)

        return list(solveset(function, x))[0]
    else: #ter mais ou menos de uma variáveis
        print('Equação sem variável ou com mais de uma.')
        pass

def compress_ts(temperatura=(298., 298.), pressao=(101235., 101235.), coef_adiabatico=5/3, rendimento=1.) -> float:
    #ESSA FUNÇÃO SÓ PERMITE UMA VÁRIAVEL DE CADA VEZ, CASO SEJAM COLOCADOS DOIS X A FUNÇÃO IRÁ FALHAR (isso é intencional)
    from sympy import symbols, solveset
    variaveis = [*temperatura, *pressao, coef_adiabatico, rendimento] #lista de variaveis
              # [(0, 1), (2, 3), 4, 5] ordem dos valores da lista de variaveis
    x = symbols('x')
    
    cont = 0
    for i, j in enumerate(variaveis): #procura qual é a variável e qual a sua posição na lista de variáveis
        if j == 'x': variaveis[i], cont = x, cont+1
    
    if cont == 1: #checa se existe apenas uma variável
        function = variaveis[1] - variaveis[0]*(1 + ((variaveis[3]/variaveis[2])**((variaveis[4]-1)/variaveis[4]) - 1)/variaveis[5])

        return list(solveset(function, x))[0]
    else: #ter mais ou menos de uma variáveis
        print('Equação sem variável ou com mais de uma.')
        pass
    
def compress_pot(H_compressor=0, vazao=1, rendimento=1., densidade=1.2) -> float: #calcula a potência
    return densidade*vazao*H_compressor/rendimento

def compress_est(pressao=(1., 4.), compressao=4) -> float: #calcula o número de estagios necessários
    from numpy import log
    return (log(pressao[1]) - log(pressao[0]))/log(compressao)